In [1]:
from pathlib import Path
import torch
import torch.nn.functional as F
import torchvision.transforms as tvtfs
from torch.utils.data import Dataset, DataLoader
import datasets as hfds
from transformers import ViTForImageClassification, ViTImageProcessor, get_scheduler
from tqdm import tqdm

In [2]:
def load_model(model_name_or_path: str = "google/vit-base-patch16-224"):
    return ViTForImageClassification.from_pretrained(model_name_or_path, num_labels=1000)

In [3]:
class ImagenetDataset(Dataset):
    """ Imagenet dataset with random masking """
    def __init__(self, split: str, rand_masks: bool = False):
        self.split = split
        self.ds = hfds.load_dataset("ILSVRC/imagenet-1k", split=split, cache_dir="/shared_data0/antonxue/")
        self.processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
        self.rand_masks = rand_masks

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        image = item["image"]
        image = self.processor(
            image if image.mode == "RGB" else image.convert("RGB"),
            return_tensors = "pt",
        )["pixel_values"].view(3,224,224)
        # Each 16x16 patch (14x14 grid) has a random chance of turning off, randomly.
        if self.rand_masks:
            small_mask = torch.rand(1,1,14,14) < torch.rand(())
            large_mask = F.interpolate(small_mask.float(), scale_factor=(16,16))
            image = image.view(3,224,224) * large_mask.view(1,224,224)
        return {"image": image, "label": item["label"]}

In [4]:
len(ImagenetDataset(split="train"))

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for ILSVRC/imagenet-1k contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ILSVRC/imagenet-1k
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

1281167

In [5]:
len(ImagenetDataset(split="test"))

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for ILSVRC/imagenet-1k contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ILSVRC/imagenet-1k
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

100000

In [6]:
def train_one_epoch(model, dataloader, optimizer, lr_scheduler):
    model.train()
    device = next(model.parameters()).device
    all_losses, all_accs = [], []
    pbar = tqdm(dataloader)
    for batch in pbar:
        image, label = batch["image"].to(device), batch["label"].to(device)
        logits = model(image).logits
        loss = F.cross_entropy(logits, label)
        loss.backward(); optimizer.step(); optimizer.zero_grad(); lr_scheduler.step()

        all_losses.append(loss.detach().item())
        all_accs.append((logits.argmax(dim=-1) == label).float().mean().detach().item())
        pbar.set_description(
            f"lr {lr_scheduler.get_last_lr()[0]:.3e}, "
            + f"loss {torch.tensor(all_losses[-16:]).mean():.3e}, "
            + f"acc {torch.tensor(all_accs[-16:]).float().mean():.3e}"
        )
    return {"state_dict": model.state_dict(), "train_losses": all_losses, "train_accs": all_accs}

In [7]:
@torch.no_grad()
def valid_one_epoch(model, dataloader):
    model.eval()
    device = next(model.parameters()).device
    all_losses, all_accs = [], []
    pbar = tqdm(dataloader)
    for batch in pbar:
        image, label = batch["image"].to(device), batch["label"].to(device)
        logits = model(image).logits
        loss = F.cross_entropy(logits, label)

        all_losses.append(loss.detach().item())
        all_accs.append((logits.argmax(dim=-1) == label).float().mean().detach().item())
        pbar.set_description(
            f"loss {torch.tensor(all_losses[-16:]).mean():.3e}, "
            + f"acc {torch.tensor(all_accs[-16:]).float().mean():.3e}"
        )
    return {"valid_losses": all_losses, "valid_accs": all_accs}

In [8]:
def run_finetuning_epochs(
    num_epochs: int = 5,
    bsz: int = 128,
    lr: float = 1e-4,
    device: str = "cuda",
    saveto_dir: str = "../_saved_models",
):
    # Make directory if doesn't exist
    Path(saveto_dir).mkdir(parents=True, exist_ok=True)

    # Initialize dataloaders
    train_dataloader = DataLoader(ImagenetDataset("train", rand_masks=True), batch_size=bsz, num_workers=24, shuffle=True)
    valid_dataloader = DataLoader(ImagenetDataset("validation", rand_masks=True), batch_size=bsz, num_workers=24, shuffle=True)

    # Set up the model, optimizer, and learning rate scheduler
    model = load_model()
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    num_train_steps = len(train_dataloader) * num_epochs
    lr_scheduler = get_scheduler(
        name = "linear",
        optimizer = optimizer,
        num_warmup_steps = int(0.1 * num_train_steps),
        num_training_steps = num_train_steps
    )

    # Fine-tuning loop
    for epoch in range(1, num_epochs+1):
        saveto = f"{saveto_dir}/google_vit_patch16_img224_bsz{bsz}_lr{lr:.4f}_epoch{epoch}.pt" 
        print("Training", saveto)
        train_ret = train_one_epoch(model, train_dataloader, optimizer, lr_scheduler)
        valid_ret = valid_one_epoch(model, valid_dataloader)
        train_ret["valid_losses"] = valid_ret["valid_losses"]
        train_ret["valid_accs"] = valid_ret["valid_accs"]
        torch.save(train_ret, saveto)

    return train_ret

In [ ]:
run_finetuning_epochs(num_epochs=5)

Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

Training ../_saved_models/google_vit_patch16_img224_bsz128_lr0.0001_epoch1.pt


lr 7.592e-07, loss 1.996e+00, acc 6.162e-01:   0%|                                | 38/10010 [03:02<2:57:59,  1.07s/it]/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:648: UserWarning: Metadata Warning, tag 274 had too many entries: 4, expected 1
  warnings.warn(
lr 2.402e-05, loss 1.154e+00, acc 7.280e-01:  12%|███▎                        | 1202/10010 [1:12:39<2:57:33,  1.21s/it]/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
lr 4.230e-05, loss 1.172e+00, acc 7.373e-01:  21%|█████▉                      | 2117/10010 [1:58:02<1:55:36,  1.14it/s]/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
lr 6.048e-05, loss 1.246e+00, acc 7.212e-01:  30%|████████▍                   | 3027/10010 [2:45:58<2:20:26,  1.21s/it]/opt/conda/lib/python3.1